# 네이버 월간 검색량 조회 프로그램

## 프로그램 설명
네이버 광고 API를 사용하여 키워드의 월간 검색량을 조회하는 프로그램입니다. Tkinter를 활용한 GUI 인터페이스를 제공합니다.

## 주요 기능
- 네이버 광고 API를 통한 월간 검색량 조회
- PC/모바일/전체 검색량 구분
- GUI 인터페이스 제공
- 복수 키워드 동시 조회 가능

## 사용 방법
1. 프로그램 실행 (GUI 창 열림)
2. 네이버 API 키, 시크릿 키, 고객 ID 입력
3. 조회할 키워드 입력
4. 검색량 조회 버튼 클릭
5. 결과 확인

## 필요 라이브러리
- requests: API 요청
- tkinter: GUI 인터페이스
- hmac, hashlib, base64: API 인증

## 주의사항
- 네이버 광고 API 키가 필요합니다
- API 사용량 제한이 있을 수 있습니다

In [1]:
# 네이버 광고 API를 이용하여 월간검색량 구하기

import tkinter as tk
from tkinter import messagebox, ttk
import tkinter.font as tkfont
import requests
import hmac
import hashlib
import base64
import time
from typing import Dict, List, Tuple
from dataclasses import dataclass

@dataclass
class NaverAPIConfig:
    API_KEY: str
    SECRET_KEY: str
    CUSTOMER_ID: str

@dataclass
class SearchVolume:
    pc: int
    mobile: int
    total: int

class NaverSearchVolumeApp:
    def __init__(self, root: tk.Tk, config: NaverAPIConfig):
        self.root = root
        self.config = config
        self.search_volumes = {}
        self.setup_ui()

    def setup_ui(self):
        self.root.title("네이버 검색어 조회")
        self.root.geometry("800x600")

        # Input frame
        input_frame = ttk.Frame(self.root, padding="10")
        input_frame.pack(fill=tk.X)

        entry_label = ttk.Label(input_frame, text="검색어 입력 (콤마로 구분):")
        entry_label.pack(side=tk.LEFT, padx=(0, 10))

        self.entry = ttk.Entry(input_frame, width=50)
        self.entry.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=(0, 10))

        search_button = ttk.Button(input_frame, text="검색", command=self.on_search_click)
        search_button.pack(side=tk.LEFT)

        # Sorting frame
        sort_frame = ttk.Frame(self.root, padding="10")
        sort_frame.pack(fill=tk.X)

        sort_label = ttk.Label(sort_frame, text="정렬:")
        sort_label.pack(side=tk.LEFT, padx=(0, 10))

        self.sort_column = tk.StringVar(value="Total")
        sort_column_combo = ttk.Combobox(sort_frame, textvariable=self.sort_column, 
                                         values=["Keyword", "PC", "Mobile", "Total"], 
                                         state="readonly", width=10)
        sort_column_combo.pack(side=tk.LEFT, padx=(0, 10))

        self.sort_order = tk.StringVar(value="descending")
        sort_order_combo = ttk.Combobox(sort_frame, textvariable=self.sort_order, 
                                        values=["ascending", "descending"], 
                                        state="readonly", width=10)
        sort_order_combo.pack(side=tk.LEFT, padx=(0, 10))

        sort_button = ttk.Button(sort_frame, text="정렬", command=self.sort_results)
        sort_button.pack(side=tk.LEFT)

        # Treeview for table-like display
        self.tree = ttk.Treeview(self.root, columns=('Keyword', 'PC', 'Mobile', 'Total'), show='headings')
        self.tree.heading('Keyword', text='검색어')
        self.tree.heading('PC', text='PC 검색량')
        self.tree.heading('Mobile', text='모바일 검색량')
        self.tree.heading('Total', text='총 검색량')
        self.tree.pack(pady=10, fill=tk.BOTH, expand=True)

        # Scrollbar for Treeview
        scrollbar = ttk.Scrollbar(self.root, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Set initial column widths
        for col in ('Keyword', 'PC', 'Mobile', 'Total'):
            self.tree.column(col, width=100)

        # Close button
        close_button = ttk.Button(self.root, text="프로그램 종료", command=self.quit_program)
        close_button.pack(pady=10)

    def quit_program(self):
        """프로그램을 안전하게 종료하는 메서드"""
        self.root.quit()
        self.root.destroy()

    def on_search_click(self):
        keywords = [k.strip() for k in self.entry.get().split(',') if k.strip()]
        if not keywords:
            messagebox.showwarning("입력 오류", "검색어를 입력하세요.")
            return
        
        self.search_volumes = self.get_naver_search_volume(keywords)
        self.display_search_results()

    def get_naver_search_volume(self, keywords: List[str]) -> Dict[str, SearchVolume]:
        result = {}
        for i in range(0, len(keywords), 5):
            sub_keywords = keywords[i:i + 5]
            timestamp = str(int(time.time() * 1000))
            method = "GET"
            uri = "/keywordstool"
            signature = self.generate_signature(timestamp, method, uri)

            url = "https://api.naver.com" + uri
            headers = {
                "X-API-KEY": self.config.API_KEY,
                "X-CUSTOMER": self.config.CUSTOMER_ID,
                "X-Timestamp": timestamp,
                "X-Signature": signature,
                "Content-Type": "application/json",
                "Accept": "*/*"
            }
            params = {
                "hintKeywords": ",".join(sub_keywords),
                "showDetail": "1"
            }

            try:
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()
                data = response.json()
                for item in data.get("keywordList", []):
                    keyword, search_data = self.process_search_data(item, sub_keywords)
                    if keyword:
                        result[keyword] = search_data
            except requests.RequestException as e:
                messagebox.showerror("Error", f"API 요청 중 오류 발생: {str(e)}")

        return result

    def generate_signature(self, timestamp: str, method: str, uri: str) -> str:
        message = f"{timestamp}.{method}.{uri}"
        signature = hmac.new(self.config.SECRET_KEY.encode(), message.encode(), hashlib.sha256).digest()
        return base64.b64encode(signature).decode()

    def process_search_data(self, item: Dict, sub_keywords: List[str]) -> Tuple[str, SearchVolume]:
        keyword = item.get("relKeyword")
        if keyword in sub_keywords:
            pc_search_count = self.parse_search_count(item.get("monthlyPcQcCnt", 0))
            mobile_search_count = self.parse_search_count(item.get("monthlyMobileQcCnt", 0))
            total_search_volume = pc_search_count + mobile_search_count
            return keyword, SearchVolume(pc_search_count, mobile_search_count, total_search_volume)
        return None, None

    @staticmethod
    def parse_search_count(count) -> int:
        if isinstance(count, str) and '<' in count:
            return 5
        return int(count)

    def display_search_results(self):
        # Clear previous results
        for i in self.tree.get_children():
            self.tree.delete(i)

        if not self.search_volumes:
            messagebox.showinfo("결과 없음", "검색 결과가 없습니다.")
            return
        
        # Insert new results
        for keyword, volumes in self.search_volumes.items():
            self.tree.insert('', tk.END, values=(
                keyword, 
                f"{volumes.pc:,}", 
                f"{volumes.mobile:,}", 
                f"{volumes.total:,}"
            ))

        # Adjust column widths
        for col in ('Keyword', 'PC', 'Mobile', 'Total'):
            self.tree.column(col, width=tkfont.Font().measure(self.tree.heading(col)['text']) + 20)

        # Adjust the width of the 'Keyword' column based on the longest keyword
        max_keyword_width = max(tkfont.Font().measure(keyword) for keyword in self.search_volumes.keys())
        self.tree.column('Keyword', width=max_keyword_width + 20)

    def sort_results(self):
        column = self.sort_column.get()
        order = self.sort_order.get()

        # Clear previous results
        for i in self.tree.get_children():
            self.tree.delete(i)

        # Sort the results
        sorted_results = sorted(
            self.search_volumes.items(),
            key=lambda x: getattr(x[1], column.lower()) if column != 'Keyword' else x[0],
            reverse=(order == 'descending')
        )

        # Insert sorted results
        for keyword, volumes in sorted_results:
            self.tree.insert('', tk.END, values=(
                keyword, 
                f"{volumes.pc:,}", 
                f"{volumes.mobile:,}", 
                f"{volumes.total:,}"
            ))

def main():
    config = NaverAPIConfig(
        API_KEY="0100000000f7219646b1189747046631aa4d35e62954247e750a216e37f7072b5174cbe5c2",
        SECRET_KEY="AQAAAAD3IZZGsRiXRwRmMapNNeYpncLjvqyOrRpn2cZDTV49Ug==",
        CUSTOMER_ID="373124"
    )
    root = tk.Tk()
    app = NaverSearchVolumeApp(root, config)
    root.mainloop()

if __name__ == "__main__":
    main()